# `JuMP.jl`: A Brief Initial Introduction
Here we present an initial introduction to using `JuMP.jl`. We will focus on the core modeling aspects as motivated by a simple linear programming problem. Note that this content takes inspiration from https://jump.dev/JuMP.jl/stable/tutorials/getting_started/getting_started_with_JuMP/.

## Resources
We will not be able to cover all of `JuMP.jl`'s capabilities today. Good references are:
- The tutorials, examples, manuals, and guides in `JuMP.jl`'s documentation: https://jump.dev/JuMP.jl/stable/
- The Julia optimization forum: https://discourse.julialang.org/c/domain/opt/13
- Julia Programming for Operations Research 2/e (not always up-to-date): https://www.softcover.io/read/7b8eb7d0/juliabook2/introduction

## Installation
This short course project already has `JuMP.jl` installed with the necessary solver packages. For reference, let's consider how to add `JuMP` and appropriate solvers to a project environment:

```julia
julia> cd("PATH/MyProject")

julia> ]

(@v1.7) pkg> activate .

(@MyProject) pkg> add JuMP, HiGHS, Ipopt
```
Here `HiGHS` acts an appropriate LP solver and `Ipopt` acts as an appropriate NLP solver. The list of supported solvers and the problems types they can solve is provided at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.

## Motivating Example
Consider the following linear program (LP):
$$
\begin{aligned}
& \min && 12x + 20y \\
& \;\;\text{s.t.} && 6x + 8y \geq 100 \\
&&& 7x + 12y \geq 120 \\
&&& x \geq 0 \\
&&& y \in [0, 3] \\
\end{aligned}
$$
Let's formulate this problem in `JuMP.jl` and use the HiGHS solver:

In [1]:
using JuMP, HiGHS

model = Model(HiGHS.Optimizer)

@variable(model, x >= 0)
@variable(model, 0 <= y <= 3)

@objective(model, Min, 12x + 20y)

@constraint(model, c1, 6x + 8y >= 100)
@constraint(model, c2, 7x + 12y >= 120)

print(model)

latex_formulation(model)

Min 12 x + 20 y
Subject to
 c1 : 6 x + 8 y >= 100.0
 c2 : 7 x + 12 y >= 120.0
 x >= 0.0
 y >= 0.0
 y <= 3.0


$$ \begin{aligned}
\min\quad & 12 x + 20 y\\
\text{Subject to} \quad & 6 x + 8 y \geq 100.0\\
 & 7 x + 12 y \geq 120.0\\
 & x \geq 0.0\\
 & y \geq 0.0\\
 & y \leq 3.0\\
\end{aligned} $$

That's all we have to do formulate the model and view it. Now let's optimize it!

In [2]:
optimize!(model)

@show termination_status(model)
@show primal_status(model)
@show dual_status(model)
@show objective_value(model)
@show value(x)
@show value(y)
@show shadow_price(c1)
@show shadow_price(c2);

Presolving model
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Presolve : Reductions: rows 2(-0); columns 2(-0); elements 4(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(220) 0s
          2     2.0500000000e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  2.0500000000e+02
HiGHS run time      :          0.01
termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 205.0
value(x) = 15.0
value(y) = 1.25
shadow_price(c1) = -0.25
shadow_price(c2) = -1.5


That was pretty easy and all the results can be queried with a solver independent API. Below let's go over what we just did, step by step.

## Package Importing
To write `JuMP.jl` programs, we'll need to import `JuMP` and an appropriate solver package to solve to the model. Hence, in this case we import `JuMP` and `HiGHS`: 

In [3]:
using JuMP, HiGHS

## `JuMP.jl` Models
`JuMP` builds problems incrementally in a `Model` object. Create a model by passing an optimizer to the `Model` function:

In [4]:
model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

Here, the convention for the optimizer input is `SolverName.Optimizer`.

## Decision Variables
`JuMP.jl`'s modeling API principally uses macros to provide an intuitive symbolic interface. For adding/creating optimization variables, we use the `@variable` macro. To define, $x \geq 0$ we write:

In [5]:
@variable(model, x >= 0)

x

To add $0 \leq y \leq 30$, we can write:

In [6]:
@variable(model, 0 ≤ y ≤ 30)

y

Notice that we used `≤` (from `\leq` and pressing [TAB]) instead of `<=` to highlight how we can use unicode characters instead if we prefer.

## Objective
The objective function is specified via `@objective`. Hence, to set $\text{min} \; 12x + 20y$ we write:

In [7]:
@objective(model, Min, 12x + 20y)

12 x + 20 y

## Constraints
Constraints are added via `@constraint`. Here, we name our constraints `c1` and `c2` for convenience in querying results later on (this is optional and the argument can be omitted if wanted).

In [8]:
@constraint(model, c1, 6x + 8y >= 100)

c1 : 6 x + 8 y >= 100.0

In [9]:
@constraint(model, c2, 7x + 12y >= 120)

c2 : 7 x + 12 y >= 120.0

## Printing the Model
Simply showing the model results in a summary of what components it has:

In [10]:
model

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: c1, c2, x, y

We have model with 2 optimization variables, an affine minimization objective, and 5 constraints of three different types. 

More conveniently we can print the model using `print`:

In [11]:
print(model)

Min 12 x + 20 y
Subject to
 c1 : 6 x + 8 y >= 100.0
 c2 : 7 x + 12 y >= 120.0
 x >= 0.0
 y >= 0.0
 y <= 30.0


That is certainly more human-readable. Since, we are using a Jupyter notebook, we can even print the latex formulation of our model using `latex_formulation`:

In [12]:
latex_formulation(model)

$$ \begin{aligned}
\min\quad & 12 x + 20 y\\
\text{Subject to} \quad & 6 x + 8 y \geq 100.0\\
 & 7 x + 12 y \geq 120.0\\
 & x \geq 0.0\\
 & y \geq 0.0\\
 & y \leq 30.0\\
\end{aligned} $$

## Optimization
Now that we have a model, let's optimize it using `optimize!`:

In [13]:
optimize!(model)

Presolving model
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Presolve : Reductions: rows 2(-0); columns 2(-0); elements 4(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(220) 0s
          2     2.0500000000e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  2.0500000000e+02
HiGHS run time      :          0.00


We will review methods later to specify solver options. One common one is `set_silent` which turns off the raw solver output:

In [14]:
set_silent(model)
optimize!(model)

## Querying Results
Our model is now optimized, so let's see what happened using `JuMP.jl`'s general purpose query API. 

We can see the final status of the solver (i.e., why it stopped) using `termination_status`:

In [15]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

Here, it stopped because it found the optimal solution. For a list of the possible statuses see https://jump.dev/JuMP.jl/stable/moi/reference/models/#MathOptInterface.TerminationStatusCode.

We can also check whether the solver found a primal feasible point using `primal_status`:

In [16]:
primal_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

It did find a feasible point. We can make the same check for the dual problem via `dual_status`:

In [17]:
dual_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

We also found a dual feasible point. The list of possible statuses is provided at https://jump.dev/JuMP.jl/stable/moi/reference/models/#MathOptInterface.ResultStatusCode.

Now we know that we have an optimal solution with feasible primal and dual solutions that we can interrogate.

Query the objective value via `objective_value`:

In [18]:
objective_value(model)

205.0

Now find the variable values using `value`:

In [19]:
@show value(x)
@show value(y);

value(x) = 15.0
value(y) = 1.25


Finally, we can learn about the dual solution using `shadow_price`:

In [20]:
@show value(c1)
@show value(c2);

value(c1) = 100.0
value(c2) = 120.0


We could have instead used `dual` to get the raw dual values, but `shadow_price` corrects the signs in accordance with the objective sense to have a consistent interpretation.

## Exercise: Simple QP Model
**Problem**
- Solve the following model using `JuMP.jl` using the `HiGHS` solver

$$
\begin{aligned}
& \min && 3x^2 + 2y^2 - 4x \\
& \;\;\text{s.t.} && 6x - 8y \geq 100 \\
&&& x + 12y = 120 \\
&&& x \geq 0 \\
&&& y \in [0, 3] \\
\end{aligned}
$$

In [22]:
# PUT CODE HERE
model = Model(HiGHS.Optimizer)
@variable(model, x ≥ 0)
@variable(model, 0 ≤ y ≤ 3)
@objective(model, Min, 3x^2 + 2y^2 - 4x)
@constraint(model, 6x - 8y ≥ 100)
@constraint(model, x + 12y == 120)
optimize!(model)
@show value(x)
@show value(y);

0, 20850.000353, 0, 0.000000, 0.000000, 0, 0.000000, 0.000000
1, 20850.000353, 0, 0.000000, 0.000000, 0, 0.000000, 0.000000
Model   status      : Optimal
QP ASM    iterations: 1
Objective value     :  2.0850000000e+04
HiGHS run time      :          0.00
value(x) = 84.0
value(y) = 3.0
